In [2]:
from keras.utils import to_categorical
from tensorflow.keras.utils import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

2025-05-28 11:23:32.935339: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748431413.149536      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748431413.218396      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
TRAIN_DIR = '/kaggle/input/face-expression-recognition-dataset/images/train'
TEST_DIR = '/kaggle/input/face-expression-recognition-dataset/images/validation'

In [4]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [5]:


train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)



surprise completed
fear completed
angry completed
neutral completed
sad completed
disgust completed
happy completed


In [6]:
print(train)

                                                   image     label
0      /kaggle/input/face-expression-recognition-data...  surprise
1      /kaggle/input/face-expression-recognition-data...  surprise
2      /kaggle/input/face-expression-recognition-data...  surprise
3      /kaggle/input/face-expression-recognition-data...  surprise
4      /kaggle/input/face-expression-recognition-data...  surprise
...                                                  ...       ...
28816  /kaggle/input/face-expression-recognition-data...     happy
28817  /kaggle/input/face-expression-recognition-data...     happy
28818  /kaggle/input/face-expression-recognition-data...     happy
28819  /kaggle/input/face-expression-recognition-data...     happy
28820  /kaggle/input/face-expression-recognition-data...     happy

[28821 rows x 2 columns]


In [7]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

surprise completed
fear completed
angry completed
neutral completed
sad completed
disgust completed
happy completed


In [8]:
print(test)
print(test['image'])

                                                  image     label
0     /kaggle/input/face-expression-recognition-data...  surprise
1     /kaggle/input/face-expression-recognition-data...  surprise
2     /kaggle/input/face-expression-recognition-data...  surprise
3     /kaggle/input/face-expression-recognition-data...  surprise
4     /kaggle/input/face-expression-recognition-data...  surprise
...                                                 ...       ...
7061  /kaggle/input/face-expression-recognition-data...     happy
7062  /kaggle/input/face-expression-recognition-data...     happy
7063  /kaggle/input/face-expression-recognition-data...     happy
7064  /kaggle/input/face-expression-recognition-data...     happy
7065  /kaggle/input/face-expression-recognition-data...     happy

[7066 rows x 2 columns]
0       /kaggle/input/face-expression-recognition-data...
1       /kaggle/input/face-expression-recognition-data...
2       /kaggle/input/face-expression-recognition-data...
3       /

In [9]:
from tqdm import tqdm

def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,color_mode = 'grayscale' )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features


train_features = extract_features(train['image']) 


100%|██████████| 28821/28821 [04:10<00:00, 115.28it/s]


In [10]:


test_features = extract_features(test['image'])



100%|██████████| 7066/7066 [01:01<00:00, 114.57it/s]


In [11]:
x_train = train_features/255.0
x_test = test_features/255.0

In [12]:
from sklearn.preprocessing import LabelEncoder



le = LabelEncoder()
le.fit(train['label'])

y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [13]:


y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)



In [14]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1748431739.892985      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1748431739.893679      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test)) 

Epoch 1/100


I0000 00:00:1748431745.577405      96 service.cc:148] XLA service 0x2848d190 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748431745.578229      96 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1748431745.578249      96 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1748431746.011953      96 cuda_dnn.cc:529] Loaded cuDNN version 90300


  3/226 ━━━━━━━━━━━━━━━━━━━━ 14s 63ms/step - accuracy: 0.1094 - loss: 1.9833

I0000 00:00:1748431754.747815      96 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


226/226 ━━━━━━━━━━━━━━━━━━━━ 31s 81ms/step - accuracy: 0.2325 - loss: 1.8408 - val_accuracy: 0.2583 - val_loss: 1.8106
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 10s 46ms/step - accuracy: 0.2461 - loss: 1.8108 - val_accuracy: 0.2707 - val_loss: 1.7681
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 10s 46ms/step - accuracy: 0.2872 - loss: 1.7287 - val_accuracy: 0.3487 - val_loss: 1.6300
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 11s 47ms/step - accuracy: 0.3597 - loss: 1.6160 - val_accuracy: 0.4483 - val_loss: 1.4409
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 11s 47ms/step - accuracy: 0.4096 - loss: 1.5047 - val_accuracy: 0.4764 - val_loss: 1.3605
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 11s 48ms/step - accuracy: 0.4389 - loss: 1.4453 - val_accuracy: 0.4958 - val_loss: 1.3316
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 11s 48ms/step - accuracy: 0.4617 - loss: 1.3879 - val_accuracy: 0.5088 - val_loss: 1.2707
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 11s 49ms/step - accuracy: 0.4760 - loss: 1.3611 - val_

In [17]:
from keras.models import model_from_json



model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")



json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")




